<a href="https://colab.research.google.com/github/AIWintermuteAI/aXeleRate/blob/master/resources/aXeleRate_test_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Detection model Training and Inference

In this notebook we will use axelerate, Keras-based framework for AI on the edge, to quickly setup model training and then after training session is completed convert it to .tflite and .kmodel formats.

First, let's take care of some administrative details. 

1) Before we do anything, make sure you have choosen GPU as Runtime type (in Runtime - > Change Runtime type).

2) We need to mount Google Drive for saving our model checkpoints and final converted model(s). Press on Mount Google Drive button in Files tab on your left. 

In the next cell we clone axelerate Github repository and import it. 

**It is possible to use pip install or python setup.py install, but in that case you will need to restart the enironment.** Since I'm trying to make the process as streamlined as possibile I'm using sys.path.append for import.

In [1]:
#!git clone https://github.com/AIWintermuteAI/aXeleRate.git
!cd aXeleRate && git checkout dev
import sys
sys.path.append('aXeleRate')
from axelerate import setup_training,setup_inference

M	axelerate/networks/common_utils/convert.py
M	axelerate/networks/yolo/backend/utils/augment.py
Already on 'dev'
Your branch is up to date with 'origin/dev'.


Using TensorFlow backend.


At this step you typically need to get the dataset. You can use !wget command to download it from somewhere on the Internet or !cp to copy from My Drive as in this example
```
!cp -r /content/drive/'My Drive'/pascal_20_segmentation.zip .
!unzip --qq pascal_20_segmentation.zip
```
For this notebook small test dataset is already in axelerate/sample_datasets folder, so no need to download anything.

Let's visualize our detection model test dataset. There are two images in train folder with corresponding annotations in PASCAL-VOC format in annotations folder.


In [ ]:
from axelerate.networks.yolo.backend.utils.augment import visualize_dataset

visualize_dataset(img_folder='aXeleRate/sample_datasets/detector/imgs', ann_folder='aXeleRate/sample_datasets/detector/anns', img_size=None, jitter=None)

Next step is defining a config dictionary. Most lines are self-explanatory.

Type is model frontend - Classifier, Detector or Segnet

Architecture is model backend (feature extractor) 

- Full Yolo
- Tiny Yolo
- MobileNet1_0
- MobileNet7_5 
- MobileNet5_0 
- MobileNet2_5 
- SqueezeNet
- VGG16
- ResNet50

For more information on anchors, please read here
https://github.com/pjreddie/darknet/issues/568

Labels are labels present in your dataset.
IMPORTANT: Please, list all the labels present in the dataset.

object_scale determines how much to penalize wrong prediction of confidence of object predictors

no_object_scale determines how much to penalize wrong prediction of confidence of non-object predictors

coord_scale determines how much to penalize wrong position and size predictions (x, y, w, h)

class_scale determines how much to penalize wrong class prediction

In [ ]:
config = {
        "model":{
            "type":                 "Detector",
            "architecture":         "MobileNet7_5",
            "input_size":           224,
            "anchors":              [0.57273, 0.677385, 1.87446, 2.06253, 3.33843, 5.47434, 7.88282, 3.52778, 9.77052, 9.16828],
            "labels":               ["aeroplane","person","diningtable","bottle","bird","bus","boat","cow","sheep","train"],
            "coord_scale" : 		1.0,
            "class_scale" : 		1.0,
            "object_scale" : 		5.0,
            "no_object_scale" : 	1.0
        },
        "weights" : {
            "full":   				"",
            "backend":   		    "imagenet"
        },
        "train" : {
            "actual_epoch":         1,
            "train_image_folder":   "aXeleRate/sample_datasets/detector/imgs",
            "train_annot_folder":   "aXeleRate/sample_datasets/detector/anns",
            "train_times":          4,
            "valid_image_folder":   "aXeleRate/sample_datasets/detector/imgs_validation",
            "valid_annot_folder":   "aXeleRate/sample_datasets/detector/anns_validation",
            "valid_times":          4,
            "valid_metric":         "mAP",
            "batch_size":           4,
            "learning_rate":        1e-4,
            "saved_folder":   		"detector",
            "first_trainable_layer": "",
            "augumentation":				True,
            "is_only_detect" : 		False
        },
        "converter" : {
            "type":   				["k210"]
        }
    }

Let's check what GPU we have been assigned in this Colab session, if any.

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

Finally we start the training by passing config dictionary we have defined earlier to setup_training function. The function will start the training with Checkpoint, Reduce Learning Rate on Plateau and Early Stopping callbacks. After the training has stopped, it will convert the best model into the format you have specified in config and save it to the project folder.

In [ ]:
model_path = setup_training(config_dict=config)

After training it is good to check the actual perfomance of your model by doing inference on your validation dataset and visualizing results. This is exactly what next block does. Obviously since our model has only trained on a few images the results are far from stellar, but if you have a good dataset, you'll have better results.

In [2]:
from axelerate.networks.common_utils.convert import Converter
converter = Converter('edgetpu', 'MobileNet7_5', 'aXeleRate/sample_datasets/detector/imgs_validation')
converter.convert_model('YOLO_best_mAP.h5')

Edge TPU Converter ready


/home/ubuntu/miniconda3/envs/notebook/lib/python3.7/site-packages/tensorflow_core/lite/python/lite.py:854: UserWarning: Property target_ops is deprecated, please use target_spec.supported_ops instead.
  "target_spec.supported_ops instead." % name)


/home/ubuntu/github/aXeleRate-dev/resources
edgetpu_compiler --out_dir /home/ubuntu/github/aXeleRate-dev/resources /home/ubuntu/github/aXeleRate-dev/resources/YOLO_best_mAP.tflite
Edge TPU Compiler version 2.1.302470888



Model compiled successfully in 508 ms.



Input model: /home/ubuntu/github/aXeleRate-dev/resources/YOLO_best_mAP.tflite

Input size: 1.97MiB

Output model: /home/ubuntu/github/aXeleRate-dev/resources/YOLO_best_mAP_edgetpu.tflite

Output size: 2.03MiB

On-chip memory used for caching model parameters: 1.96MiB

On-chip memory remaining for caching model parameters: 5.85MiB

Off-chip memory used for streaming uncached model parameters: 0.00B

Number of Edge TPU subgraphs: 1

Total number of operations: 36

Operation log: /home/ubuntu/github/aXeleRate-dev/resources/YOLO_best_mAP_edgetpu.log



Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider upda

In [ ]:
from axelerate.networks.common_utils.convert import Converter
converter = Converter('k210', 'MobileNet7_5', 'aXeleRate/sample_datasets/detector/imgs_validation')
converter.convert_model('YOLO_best_mAP.h5')

In [ ]:
from axelerate.networks.common_utils.convert import Converter
converter = Converter('openvino', 'MobileNet7_5', 'aXeleRate/sample_datasets/detector/imgs_validation')
converter.convert_model('YOLO_best_mAP.h5')

In [ ]:
from keras import backend as K 
K.clear_session()
setup_inference(config, model_path)

Good luck and happy training! Have a look at these articles, that would allow you to get the most of Google Colab or connect to local runtime if there are no GPUs available;

https://medium.com/@oribarel/getting-the-most-out-of-your-google-colab-2b0585f82403

https://research.google.com/colaboratory/local-runtimes.html